# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
# import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
from data_processing import DataProcessing
from clean_predictions import PredictionDataCleaner

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/detraviousjamaribrinkley/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [4]:
load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.9796735644340515

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8603723049163818

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8691261410713196

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8487552404403687

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8647665977478027

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8541026711463928

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8756679892539978



/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_80493/312081125.py:18: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


In [5]:
# pom_mappings

In [6]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.979674
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.860372
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.869126
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.848755
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.864767
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.854103
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.875668


Comparison
1. No focus on #s, more on words

In [7]:
prediction_doc = load_nlp_model("10")
observation_doc = load_nlp_model("10.5")
observation_doc_2 = load_nlp_model("11")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_80493/1472597426.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.905285656452179

In [8]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_80493/4256136358.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc_2)


0.9179838299751282

In [9]:
pdv = prediction_doc.vector # 10
pdv

array([-0.51404536,  0.47212395,  2.1167736 , -0.01427092,  2.304752  ,
       -1.0668137 , -0.06749481,  0.66065943, -0.70772064,  0.23928517,
        0.09734118,  0.8460709 , -0.755668  ,  0.8960749 , -0.26032454,
        0.16389662, -1.0758107 , -0.5154206 , -0.7502026 ,  0.7043408 ,
       -0.7463641 ,  0.38031653,  0.21543202,  0.6452693 ,  0.87040865,
       -0.61593455,  1.05369   , -0.49514434, -0.4676479 , -0.17752272,
        0.7325282 ,  0.55586946,  0.88514817,  0.30887413, -1.32146   ,
        0.62450695, -0.8645719 ,  1.2338357 ,  0.06461638,  0.25332135,
       -0.19295019, -0.41858703, -0.30587524,  1.6967193 ,  0.06577613,
       -0.30093968, -1.0996366 ,  0.03003848, -0.6891021 , -0.5832361 ,
        1.6293421 , -0.47455692,  0.7497319 , -0.3446849 ,  1.0593346 ,
       -0.86161125,  0.8103587 , -1.4620582 ,  1.1137573 ,  0.6238946 ,
        0.36318153, -1.1489872 , -0.23594345, -0.6001305 , -0.8199844 ,
        0.45873585, -0.69914055, -0.17779215, -0.29248324,  0.08

In [10]:
odv_1 = observation_doc.vector # 10.5
odv_1

array([-0.9080646 ,  0.60812   ,  1.4975039 ,  0.13347982,  2.5056555 ,
       -1.0367268 , -0.41098732,  0.04090451, -0.03050857,  0.02576262,
        0.40703857,  1.0539262 , -0.93506014,  0.7434075 ,  0.5084373 ,
        0.1645258 , -0.5799797 , -0.16927415, -0.31959233,  0.2709655 ,
       -0.58717155,  0.6172501 , -0.0513019 ,  0.55981296,  0.75056744,
       -0.35704023,  0.43813536, -0.97806215,  0.02455415, -0.15885383,
        0.6677189 ,  0.34640923,  1.0869428 ,  0.5519073 , -1.3556136 ,
        0.8847004 , -1.1715791 ,  1.5448284 ,  0.19212052,  0.31458473,
        0.5867995 , -0.3479625 , -0.69754386,  1.4195383 ,  0.1293885 ,
       -0.48202336, -1.4765706 ,  0.3098717 , -0.2429516 , -0.45504564,
        1.3766425 , -0.09753078,  0.6110693 , -0.41896328,  1.7492323 ,
       -0.64795494,  0.9188087 , -1.1608398 ,  0.5957721 ,  0.5372187 ,
        0.40344074, -1.5717208 , -0.24523644, -0.7100303 , -0.36703974,
        0.3511196 , -0.46021885, -0.11563586, -0.61490935, -0.02

[Five most popular similarity measures implementation in python](https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/)

In [11]:
odv_2 = observation_doc_2.vector # 11
odv_2

array([-0.68726933,  0.44313243,  1.9566882 , -0.12224837,  3.1766698 ,
       -0.8449706 ,  0.05103832,  0.17347622, -0.68932265, -0.22568631,
        0.09421295,  0.87212455, -0.84420013,  0.8801198 ,  0.10539109,
        0.09293109, -1.2549573 , -0.29545903, -0.7306085 ,  0.30259234,
       -0.529282  ,  0.5353837 , -0.14207791,  0.6489821 ,  1.1178412 ,
       -0.7218921 ,  0.40414548, -0.61696273, -0.33863628,  0.00572854,
        0.85412717,  0.331739  ,  0.6515338 ,  0.02570403, -1.2833562 ,
        0.4394375 , -0.98294103,  1.1191581 , -0.00796512, -0.1222468 ,
       -0.5577816 , -0.6038265 , -0.36920086,  1.9369545 ,  0.40224832,
       -0.7073438 , -0.8292583 ,  0.00955082, -0.45049372, -0.701581  ,
        0.7993792 , -0.33022267,  0.6545384 , -0.37628552,  1.5513656 ,
       -0.41906103,  0.96519375, -1.3322203 ,  1.1946062 ,  0.8410806 ,
        0.14936131, -1.4467378 , -0.25206405, -0.3211716 , -0.61483   ,
        0.16205597, -0.05807042, -0.3855081 ,  0.01195437, -0.42

- Pearson and Spearman Correlations
    - 1 perfect positive relationship
    - 0 no relationship
    -  -1 perfect negative relationshp
- Reference: https://mccormickml.com/2019/11/05/GLUE/

In [12]:
from scipy import stats
p_result_1 = stats.pearsonr(pdv, odv_1)
p_result_2 = stats.pearsonr(pdv, odv_2)

s_result_1 = stats.spearmanr(pdv, odv_1)
s_result_2 = stats.spearmanr(pdv, odv_2)

p_result_1, p_result_2, s_result_1, s_result_2

(PearsonRResult(statistic=np.float32(0.9053422), pvalue=np.float64(9.874054448912602e-37)),
 PearsonRResult(statistic=np.float32(0.9180417), pvalue=np.float64(1.5265596637168872e-39)),
 SignificanceResult(statistic=np.float64(0.8960661964188823), pvalue=np.float64(6.418927061722937e-35)),
 SignificanceResult(statistic=np.float64(0.909020618556701), pvalue=np.float64(1.671555071979663e-37)))

In [13]:
prediction_doc = load_nlp_model("Apple's stock will rise by 10%..")
prediction_doc = load_nlp_model("Apple's stock rose by 10%.")
observation_doc = load_nlp_model("Apple's stock did not rise by 11%.")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_80493/3207375201.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.8370019197463989

In [15]:
# from math import*

# def square_rooted(x):

#     return round(sqrt(sum([a*a for a in x])),3)

# def cosine_similarity(x,y):

#     numerator = sum(a*b for a,b in zip(x,y))
#     denominator = square_rooted(x)*square_rooted(y)
#     return round(numerator/float(denominator),3)

# cosine_similarity(pdv, odv)

In [ ]:
# Load on sentence and NOT only word
prediction_doc = load_nlp_model("JP Morgan Chase")
observation_doc = load_nlp_model("Chase")
observation_doc_2 = load_nlp_model("11")

similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_80493/1590756390.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.905285656452179

In [ ]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example sentences
sentence1 = "Apple's stock rose by 10%."
sentence2 = "Apple's stock rose by 10.5%."

# Tokenize the sentences
tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

# Add [CLS] and [SEP] tokens
tokens = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2 + ['[SEP]']
print("Token:", tokens)

# Convert tokens to input IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)

# Display the tokens and input IDs
print("Input IDs:", input_ids)

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Convert tokens to input IDs
input_ids1 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens1)).unsqueeze(0)  # Batch size 1
input_ids2 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens2)).unsqueeze(0)  # Batch size 1

# Obtain the BERT embeddings
with torch.no_grad():
    outputs1 = model(input_ids1)
    outputs2 = model(input_ids2)
    embeddings1 = outputs1.last_hidden_state[:, 0, :]  # [CLS] token
    embeddings2 = outputs2.last_hidden_state[:, 0, :]  # [CLS] token

# Calculate similarity
similarity_score = cosine_similarity(embeddings1, embeddings2)
print("Similarity Score:", similarity_score)

In [ ]:
mappings = {
    "Prediction": ["I am predicting it will be sunny on 4 April 2025.", "Apple's stock will be rising by 10%.", "Apple's stock will be rising by 10%."],
    "Observation": ["It was sunny on 4 April 2025", "Apple's stock is rising by 900%.",  "Apple's stock rose by 900%."]
}

df = pd.DataFrame(mappings)
df


In [ ]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

clean_data = PredictionDataCleaner(df)

In [ ]:
updated_predictions_df = clean_data.lemmatize(col_name='Prediction', lem_col_name='Predictions Lem')
updated_predictions_df

In [ ]:
updated_observations_df = clean_data.lemmatize(col_name='Observation', lem_col_name='Observations Lem')
updated_observations_df

In [ ]:
lem_df = updated_predictions_df.merge(updated_observations_df)
lem_df

- Issues above are: 

    1. "'s" isn't joined to Apple
    2. It doesn't consider "will be rising" as a single future tense phrase, thus breaks it up into words (with only one future tense word of "rising")

In [ ]:
import nltk
nltk.download('wordnet')
  
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

In [ ]:
lem.lemmatize('predicting')

In [ ]:
lem.lemmatize('rising')

In [ ]:
lem.lemmatize('rocks')

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("predicting, predicts, rising, rises, rocks, rose, will be rising, is rising by.")
doc = nlp("I am predicting it will be sunny on 4 April 2025.")

words_lemmas_list = [token.lemma_ for token in doc]
print(words_lemmas_list)
